In [15]:
from foursquare import Foursquare
from keplergl import KeplerGl
import geopandas as gpd
from PIL import Image
import csv, ast
import pandas as pd
import torch

In [60]:
uberhex_8 = gpd.read_file('Geo-JSON Files/neighbourhood_info_v2_1.geojson')
# uberhex_8 = uberhex_8.assign('U:/staff-umbrella/imagesummary/data/Delft_NL/imagedb/image_1556_f.png')
# uberhex_8 = uberhex_8.assign(image='https://d1a3f4spazzrp4.cloudfront.net/kepler.gl/documentation/image25.png')
# uberhex_8['<img>'] = 'https://d1a3f4spazzrp4.cloudfront.net/kepler.gl/documentation/image25.png'
uberhex_8['<img>'] = 'U:/staff-umbrella/imagesummary/data/Delft_NL/imagedb/image_1556_f.png'
uberhex_8['<img>'] = 'TEST.png'
uberhex_8['<img>-test'] = 'https://d1a3f4spazzrp4.cloudfront.net/kepler.gl/documentation/image25.png'
wijken = gpd.read_file('Geo-JSON Files/neighbourhood_info_v3_0.geojson')
# print(uberhex_8)

In [5]:
image = Image.open('U:/staff-umbrella/imagesummary/data/Delft_NL/imagedb/image_1556_f.png')
image.show()

In [149]:
summaries = {'7': {'Cluster 5': {'selected': '52632', 'cluster': ['52616', '52617', '52620', '52621', '52624', '52625', '52628', '52631', '52632', '52635']},
                   'Cluster 3': {'selected': '52618', 'cluster': ['52618', '52619', '52623', '52629', '52633', '52634', '52269', '52271', '52273', '52281', '52285']},
                   'Cluster 4': {'selected': '52283', 'cluster': ['52622', '52626', '52627', '52275', '52277', '52279', '52283', '52287']},
                   'Cluster 2': {'selected': '52270', 'cluster': ['52630', '52270', '52274', '52278', '52282', '52286']},
                   'Cluster 1': {'selected': '52280', 'cluster': ['52268', '52272', '52276', '52280', '52284']}},
             '8': {'Cluster 1': {'selected': '52248', 'cluster': ['52244', '52245', '52246', '52247', '52248', '52249', '52250', '52251', '52252', '52253', '52254', '52255']}}}
# {'8': {'Cluster 1': {'selected': '52248', 'cluster': ['52244', '52245', '52246', '52247', '52248', '52249', '52250', '52251', '52252', '52253', '52254', '52255']}},
#  '7': {'Cluster 0': {'selected': '52277', 'cluster': ['52616', '52617', '52620', '52621', '52624', '52625', '52628', '52629', '52630', '52631', '52632', '52634', '52635', '52268', '52269', '52270', '52271', '52272', '52273', '52274', '52276', '52278', '52280', '52281', '52282', '52284', '52285', '52286']},
#        'Cluster 1': {'selected': '52277', 'cluster': ['52618', '52619', '52622', '52623', '52626', '52627', '52633', '52275', '52277', '52279', '52283', '52287']}}}

images = gpd.read_file('Hardcoded Images 2 Neighbourhoods.geojson')
neighbourhoods = gpd.read_file('Geo-JSON Files/neighbourhood_info_v1_0.geojson')
# neighbourhoods = gpd.read_file('Geo-JSON Files/neighbourhood_info_v3_0.geojson')
neighbourhoods['<img>-summary'] = 'Figure_1.png'

embedding_neighbourhood = dict()

for neighbourhood_id in summaries:
    image_embeddings = dict()

    with open('Hardcoded Embeddings 2 Neighbourhoods.csv', mode='r', newline='', encoding='utf-8') as csvfile:
        temp = csv.DictReader(csvfile, delimiter=';')

        for row in temp:
            image_embeddings[row['image_id']] = torch.Tensor(ast.literal_eval(row['tensor']))
    
    embedding_neighbourhood[neighbourhood_id] = image_embeddings

In [160]:
mock_data = {
	'7': (torch.Tensor([0.45, 0.9, 0.1]), 0.55),
	'8': (torch.Tensor([0.88, 0.4, 0.2]), 0.67),
	'9': (torch.Tensor([0, 1.2, 0.5]), 0.67)
}

# mock_data = {
# 	'7': (torch.Tensor([20, 100, 255]), 0.1),
# 	'8': (torch.Tensor([255, 100, 20]), 0.67),
# 	'9': (torch.Tensor([0, 100, 20]), 0.67)
# }

global_min_color = float('inf')
global_max_color = -float('inf')
global_min_opacity = float('inf')
global_max_opacity = -float('inf')

for neighbourhood_id, color_tensor in mock_data.items():
	global_min_color = torch.min(color_tensor[0]) if torch.min(color_tensor[0]) < global_min_color else global_min_color
	global_max_color = torch.max(color_tensor[0]) if torch.max(color_tensor[0]) > global_max_color else global_max_color
	global_min_opacity = color_tensor[1] if color_tensor[1] < global_min_opacity else global_min_opacity
	global_max_opacity = color_tensor[1] if color_tensor[1] > global_max_opacity else global_max_opacity

normalized_data = dict()

for neighbourhood_id, color_tensor in mock_data.items():
	normalized_tensor = (mock_data[neighbourhood_id][0] - global_min_color) / (global_max_color - global_min_color) * 255

	if global_min_opacity < 0.2:
		normalized_opacity = (mock_data[neighbourhood_id][1] * (1 - 0.2) + 0.2)
	else:
		normalized_opacity = max((mock_data[neighbourhood_id][1] - global_min_opacity) / (global_max_opacity - global_min_opacity), 0.2)
	
	normalized_data[neighbourhood_id] = (normalized_tensor, normalized_opacity)

named_neighbourhoods = dict()

if 'BU_NAAM' in neighbourhoods:
	for neighbourhood_id in mock_data:
		bu_naam = neighbourhoods.loc[(neighbourhoods['neighbourhood_id'] == int(neighbourhood_id)), 'BU_NAAM'].iloc[0]
		named_neighbourhoods[bu_naam] = normalized_data[neighbourhood_id]
else:
	named_neighbourhoods = normalized_data

print(named_neighbourhoods)

colors = {key: value[0].tolist() for key, value in mock_data.items()}
opacity = {key: value[1] for key, value in mock_data.items()}

colors = {key: value[0].tolist() for key, value in named_neighbourhoods.items()}
opacity = {key: value[1] for key, value in named_neighbourhoods.items()}

{'7': (tensor([ 95.6250, 191.2500,  21.2500]), 0.2), '8': (tensor([187.0000,  85.0000,  42.5000]), 1.0), '9': (tensor([  0.0000, 255.0000, 106.2500]), 1.0)}


In [140]:
global_min_color = float('inf')
global_max_color = -float('inf')
global_min_opacity = float('inf')
global_max_opacity = -float('inf')

mock_data1 = {
	'7': (torch.Tensor([0.45, 0.9, 0.1]), 0.55),
	'8': (torch.Tensor([0.88, 0.4, 0.2]), 0.67),
	'9': (torch.Tensor([0.45, 0.9, 0.1]), 0.2),
	'10': (torch.Tensor([0.88, 0.4, 0.2]), 1),
	'11': (torch.Tensor([0.45, 0.9, 0.1]), 0.4),
	'12': (torch.Tensor([0.88, 0.4, 0.2]), 0.2)
}

for neighbourhood_id, color_tensor in mock_data1.items():
	global_min_color = torch.min(color_tensor[0]) if torch.min(color_tensor[0]) < global_min_color else global_min_color
	global_max_color = torch.max(color_tensor[0]) if torch.max(color_tensor[0]) > global_max_color else global_max_color
	global_min_opacity = color_tensor[1] if color_tensor[1] < global_min_opacity else global_min_opacity
	global_max_opacity = color_tensor[1] if color_tensor[1] > global_max_opacity else global_max_opacity

normalized_data = dict()
for neighbourhood_id, color_tensor in mock_data1.items():
	normalized_tensor = (mock_data1[neighbourhood_id][0] - global_min_color) / (global_max_color - global_min_color) * 255
	if global_min_opacity < 0.2:
		normalized_opacity = (mock_data1[neighbourhood_id][1] * (1 - 0.2) + 0.2)
	else:
		normalized_opacity = max((mock_data1[neighbourhood_id][1] - global_min_opacity) / (global_max_opacity - global_min_opacity), 0.2)
	
	normalized_data[neighbourhood_id] = (normalized_tensor, normalized_opacity)

print(normalized_data)

{'7': (tensor([111.5625, 255.0000,   0.0000]), 0.4375), '8': (tensor([248.6250,  95.6250,  31.8750]), 0.5875), '9': (tensor([111.5625, 255.0000,   0.0000]), 0.2), '10': (tensor([248.6250,  95.6250,  31.8750]), 1.0), '11': (tensor([111.5625, 255.0000,   0.0000]), 0.25), '12': (tensor([248.6250,  95.6250,  31.8750]), 0.2)}


In [175]:
config_layers = []



config = {
	'version': 'v1',
	'config': {
		'visState': {
			'filters': [],
			'layers': [{
				'id': layer_name,
				'type': 'geojson',
				'config': {
					'dataId': layer_name,
					'label': layer_name,
					'color': colors[layer_name],
					'highlightColor': [252, 242, 26, 255],
					'columns': {
						'geojson': 'geometry'
					},
					'isVisible': True,
					'visConfig': {
						'opacity': opacity[layer_name],
						'strokeOpacity': 0.8,
						'thickness': max(min(opacity[layer_name], 0.75), 0.25),
						'strokeColor': [opacity[layer_name] * 255, opacity[layer_name] * 255, opacity[layer_name] * 255],
						'colorRange': {
							'name': 'Global Warming',
							'type': 'sequential',
							'category': 'Uber',
							'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']
						},
						'strokeColorRange': {
							'name': 'Global Warming',
							'type': 'sequential',
							'category': 'Uber',
							'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']
						},
						'radius': 10,
						'sizeRange': [0, 10],
						'radiusRange': [0, 50],
						'heightRange': [0, 500],
						'elevationScale': 5,
						'enableElevationZoomFactor': True,
						'stroked': True,
						'filled': True,
						'enable3d': False,
						'wireframe': False
					},
					'hidden': False,
					'textLabel': [{
						'field': None,
						'color': [255, 255, 255],
						'size': 18,
						'offset': [0, 0],
						'anchor': 'start',
						'alignment': 'center'
					}]
				},
				'visualChannels': {
					'colorField': None,
					'colorScale': 'quantile',
					'strokeColorField': None,
					'strokeColorScale': 'quantile',
					'sizeField': None,
					'sizeScale': 'linear',
					'heightField': None,
					'heightScale': 'linear',
					'radiusField': None,
					'radiusScale': 'linear'
				}
			} for layer_name in colors],
			'interactionConfig': {
				'tooltip': {
					'fieldsToShow': {
						layer_name: [
							{'name': 'neighbourhood_id', 'format': None},
							{'name': 'BU_NAAM', 'format': None},
							{'name': '<img>-summary', 'format': '100px'}
						]
					for layer_name in colors},
					'compareMode': False,
					'compareType': 'absolute',
					'enabled': True
				},
				'brush': {
					'size': 0.5,
					'enabled': True
				},
				'geocoder': {
					'enabled': False
				},
				'coordinate': {
					'enabled': False
				}
			},
			'layerBlending': 'normal',
			'splitMaps': [],
			'animationConfig': {
				'currentTime': None, 'speed': 1
			}
		},
		'mapState': {
            'bearing': 0,
            'dragRotate': False,
            'latitude': 51.99861374750591,
			'longitude': 4.360898979541223,
        	'pitch': 0,
            'zoom': 11.886249027093603,
            'isSplit': False,
        },
		'mapStyle': {
			'styleType': 'dark',
			'topLayerGroups': {},
			'visibleLayerGroups': {
				'label': True,
				'road': True,
				'border': False,
				'building': True,
				'water': True,
				'land': True,
				'3d building': False
			},
			'threeDBuildingColor': [9.665468314072013, 17.18305478057247, 31.1442867897876],
			'mapStyles': {}
		}
	}
}

In [152]:
neighbourhood_7 = neighbourhoods.loc[neighbourhoods['neighbourhood_id'] == 7]
neighbourhood_8 = neighbourhoods.loc[neighbourhoods['neighbourhood_id'] == 8]
wijken_9 = wijken.loc[wijken['neighbourhood_id'] == 9]

In [178]:
map_1 = KeplerGl(height=800, config=config)
for neighbourhood_id in named_neighbourhoods:
	if 'BU_NAAM' in neighbourhoods:
		map_1.add_data(neighbourhoods.loc[neighbourhoods['BU_NAAM'] == neighbourhood_id], name=neighbourhood_id)
	else:
		map_1.add_data(neighbourhoods.loc[neighbourhoods['neighbourhood_id'] == int(neighbourhood_id)], name=neighbourhood_id)
# map_1.add_data(neighbourhood_7, name='7')
# map_1.add_data(neighbourhood_8, name='8')
# map_1.add_data(wijken_9, name='9')
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '7', 'type': 'geojs…

In [179]:
map_1.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '7',
     'type': 'geojson',
     'config': {'dataId': '7',
      'label': '7',
      'color': [95.62499237060547, 191.24998474121094, 21.249998092651367],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.2,
       'strokeOpacity': 0.8,
       'thickness': 0.25,
       'strokeColor': [51, 51, 51],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0,

In [159]:
map_1.save_to_html(file_name='firsts_mapTEST.html',)

Map saved to firsts_mapTEST.html!


In [51]:
config = {
	  'version': 'v1',
		'config': {'visState': {'filters': [],
   'layers': [{'id': 'f5vbvsr',
     'type': 'geojson',
  'config': {'dataId': 'hexagons',
      'label': 'hexagons',
      'color': [130, 154, 227],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [231, 159, 213],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': True,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'hexagons': [{'name': 'image',
        'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': True},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 37.75983166248221,
   'longitude': -122.4359570593814,
   'pitch': 0,
   'zoom': 11,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [7]:
config = {'version': 'v1',
'config': {
			  'visState': {
				  'filters': [],
		  'layers': [{'id': 'f5vbvsr',
     'type': 'geojson',
     'config': {'dataId': 'hexagons',
      'label': 'hexagons',
      'color': [130, 154, 227],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [231, 159, 213],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': True,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'hexagons': [{'name': '<img>',
        'format': None},{'name': '<img>-test',
        'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': True},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 51.99893863679188,
   'longitude': 4.360895056511016,
   'pitch': 0,
   'zoom': 11,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [8]:
map_1 = KeplerGl(height=600, config=config)
# map_1.add_data(data=uberhex_8, name='hexagons')
# Show the map
map_1.add_data(data=uberhex_8, name='hexagons')
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'f5vbvsr', 'type': …

In [53]:
map_1.add_data(data=uberhex_8, name='hexagons')

In [9]:
map_1.save_to_html(file_name='firsts_map.html')

Map saved to firsts_map.html!


In [54]:
map_1.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'f5vbvsr',
     'type': 'geojson',
     'config': {'dataId': 'hexagons',
      'label': 'hexagons',
      'color': [130, 154, 227],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [231, 159, 213],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRa